# Julia Things

### Environment

First things first. Let us set up the environment with the requried packages for this notebook. We will also set the desired context (e.g. `KnetArray` for gpu), the number of epochs (`nepochs`), and the variable `fast`. This variable is used to skip checking the accuracy at every epoch. 

In [1]:
for p in ("Knet", "Plots", "Plotly.jl")
    Pkg.installed(p) == nothing && Pkg.add(p)
end

using Knet, Plots
gr()

Knet.gpu(0); # set the desired GPU to use
atype   = Array{Float64}; # atype = KnetArray{Float32} for gpu usage, Array{Float32} for cpu. 
nepochs = 10
fast    = false

println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.installed("Knet"))
println("GPU: ", readstring(`nvidia-smi --query-gpu=name --format=csv,noheader`))

OS: Linux
Julia: 0.6.0
Knet: 0.8.5+
GPU: NVS 310
TITAN X (Pascal)



### New Stuff

In this notebook we introduce the following Julia/Knet packages and functions:

* Knet's function [conv4](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.conv4): Execute convolutions or cross-correlations using filters specified with `w` over tensor `x`.
* Knet's function [pool](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.pool): Compute pooling of input values (i.e., the maximum or average of several adjacent values) to produce an output with smaller height and/or width.

## Convolutional neural networks (CNNs)

In the [previous example](../chapter03_deep-neural-networks/section2-multilayer perceptrons.ipynb), we connected the nodes of our neural networks in what seems like the simplest possible way. Every node in each layer was connected to every node in the subsequent layers. 

![](https://github.com/zackchase/mxnet-the-straight-dope/blob/master/img/multilayer-perceptron.png?raw=true)

This can require a lot of parameters! If our input were a 256x256 color image (still quite small for a photograph), and our network had 1,000 nodes in the first hidden layer, then our first weight matrix would require (256x256x3)x1000 parameters. That's nearly 200 million. Moreover the hidden layer would ignore all the spatial structure in the input image even though we know the local structure represents a powerful source of prior knowledge. 

Convolutional neural networks incorporate convolutional layers. These layers associate each of their nodes with a small window, called a *receptive field*, in the previous layer, instead of connecting to the full layer. This allows us to first learn local features via transformations that are applied in the same way for the top right corner as for the bottom left. Then we collect all this local information to predict global qualities of the image (like whether or not it depicts a dog). 

![](http://cs231n.github.io/assets/cnn/depthcol.jpeg)
(Image credit: Stanford cs231n http://cs231n.github.io/assets/cnn/depthcol.jpeg)

In short, there are two new concepts you need to grok here. First, we'll be introducting *convolutional* layers. Second, we'll be interleaving them with *pooling* layers. 

## Data

In [54]:
include(Knet.dir("data","mnist.jl"))
xtrn, ytrn, xtst, ytst = mnist()
num_samples = 1000;
batch_size  = 10;
dtrn = minibatch(xtrn[:, :, :, 1:num_samples], ytrn[1:num_samples], batch_size, xtype=atype);
dtst = minibatch(xtst[:, :, :, 1:num_samples], ytst[1:num_samples], batch_size, xtype=atype);

##  Parameters

Each node in convolutional layer is associated with a 3D block (`height` x `width` x `channel`) in the input tensor. Moreover, the convolutional layer itself has multiple output channels. So the layer is parameterized by a 4 dimensional weight tensor, commonly called a *convolutional kernel*.

The output is produced by sliding the kernel across the input image skipping locations according to a pre-defined *stride* (but we'll just assume that to be 1 in this tutorial). Let's initialize some such kernels from scratch.

In [55]:
# (kx, ky, input_size, output_size)
initweights(atype) = map(atype, Any[ xavier(Float32,3,3,1,20),  zeros(Float32,1,1,20,1),
                                     xavier(Float32,5,5,20,50), zeros(Float32,1,1,50,1),
                                     xavier(Float32,128,800),   zeros(Float32,128,1),
                                     xavier(Float32,10,128),    zeros(Float32,10,1) ]);

The shape of each kernel is ($k_x, k_y$, ch_input, ch_output), where $k_x, k_y$ are the kernel size (sliding window size), ch_input is the number of input channels (1 for gray scale, 3 for RGB), and ch_output is the number of output channels (or output filters). Basically, the number of output channels is really the number of filters we want to create. Each new filter is convolved with each input filter. For the first layer the input is the image $x$, therefore we convolve 20 filters with a single input image (for RGB, we convolved all three images). 

## Model 

We will build our `predict` function by introducing two new functions. First, we will new Knet's function [conv4](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.conv4) to execute convolutions or cross-correlations using filters specified with `w` over tensor `x`. After that we will use Knet's function [pool](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.pool). Pooling gives us a way to downsample in the spatial dimensions. Early convnets typically used average pooling, but max pooling tends to give better results. 

![](https://qph.ec.quoracdn.net/main-qimg-8afedfb2f82f279781bfefa269bc6a90)

A typical pooling layer is defined by the `window size` (typically 2x2), `stride size` (typically (2,2)), and `pooling type`. 



In [56]:
function predict(w, x)
    x = pool(relu.(conv4(w[1], x) .+ w[2]))
    x = pool(relu.(conv4(w[3], x) .+ w[4]))
    x = relu.(w[5] * mat(x) .+ w[6])
    return w[7] * x .+ w[8]
end

predict (generic function with 1 method)

In [57]:
loss(w, x, ygold, predict) = nll(predict(w, x), ygold);
lossgradient = grad(loss);

In [58]:
function train(w, dtrn, optim, predict; epochs=10)
    
    for epoch = 1:epochs
        for (x, y) in dtrn
            g = lossgradient(w, x, y, predict)
            update!(w, g, optim)
        end
    end
end

train (generic function with 1 method)

In [62]:
optim(w; lr=0.1) = optimizers(w, Sgd;  lr=lr);

In [63]:
function report(epoch, w, dtrn, dtst, predict)
    println((:epoch, epoch, :trn, accuracy(w, dtrn, predict), :tst, accuracy(w, dtst, predict)))
end

report (generic function with 1 method)

In [64]:
w   = initweights(atype);
opt = optim(w);

if fast
    train(w, dtrn, opt, predict; epochs=nepochs)
else
    for epoch = 1:nepochs
        train(w, dtrn, opt, predict; epochs=1)
        report(epoch, w, dtrn, dtst, predict)
    end
end

(:epoch, 1, :trn, 0.744, :tst, 0.712)
(:epoch, 2, :trn, 0.921, :tst, 0.868)
(:epoch, 3, :trn, 0.908, :tst, 0.867)
(:epoch, 4, :trn, 0.966, :tst, 0.899)
(:epoch, 5, :trn, 0.939, :tst, 0.872)
(:epoch, 6, :trn, 0.962, :tst, 0.894)
(:epoch, 7, :trn, 0.98, :tst, 0.912)
(:epoch, 8, :trn, 0.998, :tst, 0.93)
(:epoch, 9, :trn, 0.993, :tst, 0.935)
(:epoch, 10, :trn, 0.998, :tst, 0.934)


Not bad! consider that we only used 1000 samples, and this time around we were able to achieve 93% accuracy on the `test` dataset. 

## Conclusion

Contained in this example are nearly all the important ideas you'll need to start attacking problems in computer vision. While state-of-the-art vision systems incorporate a few more bells and whistles, they're all built on this foundation. Believe it or not, if you knew just the content in this tutorial 5 years ago, you could probably have sold a startup to a Fortune 500 company for millions of dollars. Fortunately (or unfortunately?), the world has gotten marginally more sophisticated, so we'll have to come up with some more sophisticated tutorials to follow.